In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
last_measurement=session.query(Measurement.date).order_by(Measurement.date.desc()).first()

last_measurement_result,=last_measurement

year,month,day=last_measurement_result.split("-")

one_year_query=dt.date(int(year),int(month),int(day))-dt.timedelta(days=365)

precipitation=session.query(Measurement.date, Measurement.prcp).filter(Measurement.date>=one_year_query).order_by(Measurement.date)
precipitation_df=pd.DataFrame(precipitation)

date_index_df=precipitation_df.set_index("date")
date_index_df.sort_values(["date"])
#date_index_df.head(20)
date_index_df.plot(alpha=0.75, rot=90, figsize=(15,10))
plt.xlabel("date")
plt.ylabel("precipitation")
plt.title("Rainfall")
plt.legend("precipitation")

plt.tight_layout()
plt.show()

![precipitation](Images/precipitation.png)

In [ ]:
date_index_df.describe()

![describe](Images/describe.png)

In [ ]:
stations_count=session.query(Measurement.station).group_by(Measurement.station).count()
print(stations_count)

In [ ]:
active_stations=session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc())

for rows in active_stations:
    print(rows)

In [ ]:
#tobs = temperature observations
most_active_station=active_stations[0][0]
station_stats=session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs))
print(f"Most Active Station:{most_active_station}, Lowest T:{station_stats[0][0]}, Highest T:{station_stats[0][1]}, Average T:{station_stats[0][2]}")

In [ ]:
temp_obs_count=session.query(Measurement.station, Measurement.date, Measurement.tobs).filter(Measurement.station==most_active_station).filter(Measurement.date > one_year_query).order_by(Measurement.date).all()
temp_obs_df=pd.DataFrame(temp_obs_count)
bins=12
plt.hist(temp_obs_df["tobs"], bins)
plt.ylabel("Frequency")
plt.title("Yearly TempObs for Station USC00519281")
plt.legend("tobs")
plt.tight_layout()
plt.show()

![precipitation](Images/station-histogram.png)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
previous_year_data=calc_temps("2011-02-28", "2011-03-05")
print(previous_year_data)

In [ ]:
previous_year_df=pd.DataFrame(previous_year_data, columns=["Min Temp", "Max Temp", "Avg Temp"])
average_temp=previous_year_df["Avg Temp"]
peak_to_peak=previous_year_df.iloc[0]["Max Temp"]-previous_year_df.iloc[0]["Min Temp"]
average_temp.plot(kind="bar", yerr=peak_to_peak, figsize=(6,10), alpha=0.75, color="coral")
plt.title("Trip Avg Temp")
plt.ylabel("Temp(F)")
plt.ylim(0,100)
plt.xticks([])
plt.grid(linestyle="-")
plt.show()

In [ ]:

#inspector = inspect(engine)
#inspector.get_table_names()

rainfall_stats=session.query(Measurement.station, Station.name, Station.latitude, Station.longitude, Station.elevation, func.sum(Measurement.prcp)).filter(Measurement.date.between('2011-02-28','2011-03-05')).order_by(func.sum(Measurement.prcp).desc())
for row in rainfall_stats:
    print(row)

## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
